# Machine Problem 2

Instructions: Implement the RMSProp and Adam (as discussed in the video lectures). You may use the helper functions provided below. 

## Helper Functions

In [ ]:
using ForwardDiff
using Plots
using Symbolics

### helper functions
function optimize(f, params)
    w = rand(p)
    for t in 1:N
        x = X[t, :]
    end
end

function compute_gradient(f, x)
    grad = ForwardDiff.gradient(f, x)
    return grad
end

function symbolic_2dgradient(f)

    @variables x y

    D_x = Differential(x)
    D_y = Differential(y)

    df_dx = expand_derivatives(D_x(f(x, y)))  # Partial derivative w.r.t. x
    df_dy = expand_derivatives(D_y(f(x, y)))  # Partial derivative w.r.t. y

    println("∂f/∂x: $df_dx")
    println("∂f/∂y: $df_dy")

end

### learning algorithms
function sgd_momentum(obj_func, num_params, start; γ=0.9, η=0.001, error=0.00001)

    θ = start
    history = θ
    v = zeros(num_params)
    grad = compute_gradient(obj_func, θ)

    while abs(grad[1]) > error || abs(grad[2]) > error
        grad = compute_gradient(obj_func, θ)
        v = γ .* v + -η .* grad
        θ = θ + v
        history = hcat(history, θ)
    end

    return (round.(θ, digits=3), history)
end

### plotting

# from MP 2 Sample Output (HTML)
function countour_plot_1(func, p_title, xrange, yrange, fill=false, levels=100)
    x_vals = range(xrange[1], xrange[2], length=100)
    y_vals = range(yrange[1], yrange[2], length=100)

    z_vals = [func(x, y) for y in y_vals, x in x_vals]
    Plots.contour(x_vals, y_vals, z_vals, levels=50, xlabel="x", ylabel="y", fill=fill, title=p_title)
end

function countour_plot(func, p_title, xrange, yrange, x_scatter, y_scatter, fill=false, levels=100)
    x_vals = range(xrange[1], xrange[2], length=100)
    y_vals = range(yrange[1], yrange[2], length=100)

    z_vals = [func(x, y) for y in y_vals, x in x_vals]
    contour(x_vals, y_vals, z_vals, levels=50, xlabel="x", ylabel="y", fill=fill, title=p_title)
    trajectory_plot(x_scatter, y_scatter)
end

function trajectory_plot(x_scatter, y_scatter, legend=true)
    plot!(x_scatter, y_scatter, color=:red, label=legend ? "Trajectory" : "")
    scatter!([x_scatter[1]], [y_scatter[1]], color=:black, label=legend ? "Initial Solution" : legend)
    scatter!([x_scatter[end]], [y_scatter[end]], markershape=:utriangle, color=:black, label=legend ? "Best Solution" : legend)
end

### testing

function test_sgd(sgd_func, test_func, test_solutions, p_title, num_params, start_points; kargs...)
    contours_drawn = false
    for start in start_points
        Z = sgd_func(test_func, num_params, start; kargs...)
        history = Z[2]
        if !contours_drawn
            countour_plot(test_func, p_title, [-10, 10], [-10, 10], history[1, :], history[2, :], false, 5)
            contours_drawn = true
        else
            trajectory_plot(history[1, :], history[2, :], false)
        end
    end
    scatter!(test_solutions[1], test_solutions[2], color=:blue, marker=:square, label="Best Solution")
    plot!(figsize=(60, 40), xlimits=(-11, 11), ylimits=(-11, 11))
end


### You need to implement the benchmark functions below. You may refer to this page for reference: https://en.wikipedia.org/wiki/Test_functions_for_optimization

In [ ]:
### Benchmark functions
function beale_func(u)
    x = u[1]
    y = u[2]
    return beale_func(x, y)
end

function beale_func(x, y)
    return (1.5 - x + x * y)^2 + (2.25 - x + x * y^2)^2 + (2.625 - x + x * y^3)^2
end

function booth_func(u)
    x = u[1]
    y = u[2]
    return booth_func(x, y)
end

function booth_func(x, y)
    return (x + 2 * y - 7)^2 + (2 * x + y - 5)^2
end
function matyas_func(u)
    x = u[1]
    y = u[2]
    return matyas_func(x, y)
end

function matyas_func(x, y)
    return 0.26 * (x^2 + y^2) - 0.48 * x * y
end

function himmelblau_func(u)
    x = u[1]
    y = u[2]
    return himmelblau_func(x, y)
end

function himmelblau_func(x, y)
    return (x^2 + y - 11)^2 + (x + y^2 - 7)^2
end

function mccormick_func(u)
    x = u[1]
    y = u[2]
    return mccormick_func(x, y)
end

function mccormick_func(x, y)
    return sin(x + y) + (x - y)^2 - 1.5 * x + 2.5 * y + 1
end

function goldstein_price_func(u)
    x = u[1]
    y = u[2]
    return goldstein_price_func(x, y)
end

function goldstein_price_func(x, y)
    return (1 + (x + y + 1)^2 * (19 - 14 * x + 3 * x^2 - 14 * y + 6 * x * y + 3 * y^2)) * (30 + (2 * x - 3 * y)^2 * (18 - 32 * x + 12 * x^2 + 48 * y - 36 * x * y + 27 * y^2))
end

function three_hump_camel_func(u)
    x = u[1]
    y = u[2]
    return three_hump_camel_func(x, y)
end

function three_hump_camel_func(x, y)
    return 2 * x^2 - 1.05 * x^4 + (x^6 / 6) + x * y + y^2
end


In [ ]:
TEST_FUNCTIONS = [
    Dict(
        "func" => beale_func,
        "solutions" => [[3], [0.5]],
    ),
    Dict(
        "func" => booth_func,
        "solutions" => [[1], [3]],
    ),
    Dict(
        "func" => matyas_func,
        "solutions" => [[0], [0]],
    ),
    Dict(
        "func" => himmelblau_func,
        "solutions" => [[3, -2.805118, -3.779310, 3.584428], [2, 3.131312, -3.283186, -1.848126]],
    ),
    Dict(
        "func" => mccormick_func,
        "solutions" => [[-0.54719], [-1.54719]],
    ),
    Dict(
        "func" => goldstein_price_func,
        "solutions" => [[0], [-1]],
    ),
    Dict(
        "func" => three_hump_camel_func,
        "solutions" => [[0], [0]],
    ),
];

## SGD with Momentum

For each function, determine the trajectory of the algorithm given the following starting points

In [ ]:
start_1 = [10, -10]
start_2 = [-4, 4]
start_3 = [0, 0]
start_4 = [5, 5]

plots = []
for test_function in TEST_FUNCTIONS
    func = test_function["func"]
    solutions = test_function["solutions"]
    p = test_sgd(sgd_momentum, func, solutions, string(func), 2, [start_1, start_2, start_3, start_4])
    push!(plots, p)
end

# Combine all plots into a single layout
plot(plots..., layout=(7, 1), legend=false, size=(500, 2000))

## RMSProp

In [ ]:
### Implement the RMSProp here
function rmsprop(obj_func, num_params, start; γ=0.9, η=0.001, ϵ=1e-8, error=0.00001, max_trials=100000)
    θ = start
    history = θ
    grad = compute_gradient(obj_func, θ)
    v = zeros(size(grad))
    t = 0

    while (abs(grad[1]) > error || abs(grad[2]) > error) && t < max_trials
        t += 1
        v = γ * v + (1 - γ) * grad .^ 2
        θ = θ - (η ./ (sqrt.(max.(v, 0)) .+ ϵ)) .* grad
        grad = compute_gradient(obj_func, θ)
        history = hcat(history, θ)
    end
    return (round.(θ, digits=3), history)
end

Task 1. For each function, determine the trajectory of the algorithm given the following starting points

In [ ]:
start_1 = [10, -10]
start_2 = [-4, 4]
start_3 = [0, 0]
start_4 = [5, 5]

plots = []
for test_function in TEST_FUNCTIONS
    func = test_function["func"]
    solutions = test_function["solutions"]
    p = test_sgd(rmsprop, func, solutions, string(func), 2, [start_1, start_2, start_3, start_4])
    push!(plots, p)
end

# Combine all plots into a single layout
plot(plots..., layout=(7, 1), legend=false, size=(500, 2000))

Task 2. For each function, plot the trajectory of the algorithm using four randomly generated starting points of your choice. Range of x and y must be within -10 and 10

In [ ]:
# Number of random starting points
num_starting_points = 4

# Generate random starting points within the range [-10, 10]
starting_points = [rand(-10:10, 2) for _ in 1:num_starting_points]

plots = []
for test_function in TEST_FUNCTIONS
    p = test_sgd(rmsprop, test_function["func"], test_function["solutions"], 2, starting_points)
    push!(plots, p)
end

# Combine all plots into a single layout
plot(plots..., layout=(7, 1), legend=false, size=(500, 2000))

## Adaptive Moment Estimation (Adam)

In [ ]:
### Implement the RMSProp here
function adam(obj_func, num_params, start; γ=0.9, η=0.001, β1=0.9, β2=0.999, ϵ=1e-8, error=0.00001, max_trials=100000)

    θ = start
    history = θ
    grad = compute_gradient(obj_func, θ)
    m = zeros(num_params)
    v = zeros(num_params)
    t = 0

    while (abs(grad[1]) > error || abs(grad[2]) > error) && t < max_trials
        t += 1
        m = (β1 * m) + (1 - β1) * grad
        v = (β2 * v) + (1 - β2) * (grad .^ 2)

        m_hat = m / (1 - β1^t)
        v_hat = v / (1 - β2^t)

        θ = θ - η * m_hat ./ (sqrt.(max.(v_hat, 0)) .+ ϵ)
        grad = compute_gradient(obj_func, θ)
        history = hcat(history, θ)
    end
    return (round.(θ, digits=3), history)
end

Task 1. For each function, determine the trajectory of the algorithm given the following starting points

In [ ]:
start_1 = [10, -10]
start_2 = [-4, 4]
start_3 = [0, 0]
start_4 = [5, 5]

plots = []
for test_function in TEST_FUNCTIONS
    func = test_function["func"]
    solutions = test_function["solutions"]
    p = test_sgd(adam, func, solutions, string(func), 2, [start_1, start_2, start_3, start_4])
    push!(plots, p)
end

# Combine all plots into a single layout
plot(plots..., layout=(7, 1), legend=false, size=(500, 2000))

Task 2. For each function, plot the trajectory of the algorithm using four randomly generated starting points of your choice.

In [ ]:
# Number of random starting points
num_starting_points = 4

# Generate random starting points within the range [-10, 10]
starting_points = [rand(-10:10, 2) for _ in 1:num_starting_points]

plots = []
for test_function in TEST_FUNCTIONS
    func = test_function["func"]
    solutions = test_function["solutions"]
    p = test_sgd(adam, func, solutions, string(func), 2, starting_points)
    push!(plots, p)
end

# Combine all plots into a single layout
plot(plots..., layout=(7, 1), legend=false, size=(500, 2000))